::::
:::{thebe-button}
:::
::::

# Get mean absolute error


In [ ]:
from boilercv_docs.nbs import init

paths = init()

from inspect import Signature

from matplotlib.figure import Figure
from matplotlib.pyplot import subplots
from numpy import pi
from pandas import DataFrame, Series, merge_ordered, read_hdf
from seaborn import scatterplot

from boilercv.correlations import beta
from boilercv.correlations import nusselt as correlations_nusselt
from boilercv.dimensionless_params import jakob, prandtl, thermal_diffusivity
from boilercv_docs.nbs import HIDE, set_display_options
from boilercv_pipeline.experiments.e230920_subcool import (
    GBC,
    PROCESSED_TRACKS,
    THERMAL_DATA,
)
from boilercv_pipeline.experiments.e240215_plotting import cat10

S = Series

TIME = "2023-09-20T17:14:18"
"""Timestamp of the trial to be analyzed."""
FIGURES: list[Figure] = []
"""Notebook figures available for export."""

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

# Plotting
CORRELATIONS_PALETTE = cat10
"""For plotting correlations."""
MAX_BETA_MAE = 1.1
"""Maximum mean absolute error of beta to plot."""
MAX_NUSSELT_MAE = 200
"""Maximum mean absolute error of nusselt to plot."""
TRACKS_ALPHA = 0.3
"""Transparency of the tracks."""
TRACKS_SIZE = 10
"""Size of the tracks."""

SCALE = 1.6
"""Plot scale."""
SIZE = 20 * SCALE
"""Size of the tracks."""
FONT_SCALE = SCALE
"""Font scale for plotting."""
WIDTH_SCALE = 2.00
"""Width scale for plotting."""
HEIGHT_SCALE = 1.30
"""Height scale for plotting."""

HIDE

In [ ]:
set_display_options(FONT_SCALE)
path_time = TIME.replace(":", "-")
departing_long_lived_objects: DataFrame = read_hdf(  # pyright: ignore[reportAssignmentType]
    (PROCESSED_TRACKS / f"processed_tracks_{path_time}").with_suffix(".h5")
)
thermal_data = read_hdf(THERMAL_DATA)
subcooling = thermal_data.subcool[TIME]
beta_correlations = beta.get_correlations()
nusselt_correlations = correlations_nusselt.get_correlations()
object_averages = (
    departing_long_lived_objects.set_index("bubble")
    .groupby("bubble", **GBC)
    .mean()
    .mean()
)
constants = {
    "Ja": jakob(
        liquid_density=LIQUID_DENSITY,
        vapor_density=VAPOR_DENSITY,
        liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        subcooling=subcooling,
        latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
    ),
    "Pr": prandtl(
        dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    ),
    "alpha": thermal_diffusivity(
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
        density=LIQUID_DENSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    ),
    "pi": pi,
}
beta_mae = (
    (
        departing_long_lived_objects.loc[:, ["bubble", "Re_b0", "Fo_0", "beta"]]
        .rename(columns={"beta": "experimental"})
        .set_index(["bubble"], drop=True)
    )
    .groupby("bubble", **GBC)
    .apply(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Fo_0": df["Fo_0"],
                                "Re_b0": df["Re_b0"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in beta_correlations.items()
            })
            .drop(columns=["experimental", "Re_b0", "Fo_0"])
            .sum()
        )
        / len(df)
    )
).rename_axis(index="bubble")

beta_mae

In [ ]:
nusselt_mae = (
    (
        departing_long_lived_objects.loc[
            :, ["bubble", "Re_b", "Re_b0", "Fo_0", "beta", "Nu_c"]
        ]
        .rename(columns={"Nu_c": "experimental"})
        .set_index(["bubble"], drop=True)
    )
    .groupby("bubble", **GBC)
    .apply(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Re_b": df["Re_b"],
                                "Re_b0": df["Re_b0"],
                                "Fo_0": df["Fo_0"],
                                "beta": df["beta"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in nusselt_correlations.items()
            })
            .drop(columns=["experimental", "Re_b0", "Re_b", "Fo_0", "beta"])
            .sum()
        )
        / len(df)
    )
    .rename_axis(index="bubble")
)

nusselt_mae

In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
ax.set_xlim(0, MAX_BETA_MAE)
ax.set_ylim(0, MAX_NUSSELT_MAE)
scatterplot(
    ax=ax,
    x="Beta (MAE)",
    y="Nusselt (MAE)",
    hue="Correlation",
    palette=CORRELATIONS_PALETTE,
    s=SIZE,
    data=merge_ordered(
        beta_mae.melt(
            var_name="Correlation", value_name="Beta (MAE)", ignore_index=False
        ),
        nusselt_mae.melt(
            var_name="Correlation", value_name="Nusselt (MAE)", ignore_index=False
        ),
        on=["bubble", "Correlation"],
    ),
)
figure.set_figwidth(WIDTH_SCALE * figure.get_figwidth())
figure.set_figheight(HEIGHT_SCALE * figure.get_figheight())
HIDE

In [ ]:
beta_mae_2 = (
    (
        departing_long_lived_objects.loc[:, ["Re_b0", "Fo_0", "beta"]].rename(
            columns={"beta": "experimental"}
        )
    )
    .pipe(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Re_b0": df["Re_b0"],
                                "Fo_0": df["Fo_0"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in beta_correlations.items()
            })
            .drop(columns=["experimental", "Re_b0", "Fo_0"])
            .sum()
        )
        / len(df)
    )
    .rename("Beta (MAE)")
)

beta_mae_2

In [ ]:
nusselt_mae_2 = (
    (
        departing_long_lived_objects.loc[
            :, ["Fo_0", "Re_b", "Re_b0", "beta", "Nu_c"]
        ].rename(columns={"Nu_c": "experimental"})
    )
    .pipe(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Fo_0": df["Fo_0"],
                                "Re_b": df["Re_b"],
                                "Re_b0": df["Re_b0"],
                                "beta": df["beta"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in nusselt_correlations.items()
            })
            .drop(columns=["experimental", "Fo_0", "Re_b", "Re_b0", "beta"])
            .sum()
        )
        / len(df)
    )
    .rename("Nusselt (MAE)")
)


nusselt_mae_2

In [ ]:
mae = (
    DataFrame([beta_mae_2, nusselt_mae_2])
    .T.rename_axis(index="Correlation")
    .assign(**{"subcooling": subcooling})
)

mae